In [16]:
import boto3
import pandas as pd
from io import StringIO, BytesIO
from datetime import datetime, timedelta

In [17]:
# adapter layer
def read_csv_to_df(bucket,key,decoding = 'utf-8', sep = ','):
    csv_obj = bucket.Object(key=key).get().get('Body').read().decode(decoding)
    data = StringIO(csv_obj)
    df = pd.read_csv(data, delimiter=sep)
    return df

def write_df_to_s3(bucket, df, key):
    out_buffer = BytesIO()
    df.to_parquet(out_buffer, index=False)  
    bucket.put_object(Body=out_buffer.getvalue(), Key=key)
    return True
    
def list_files_in_prefix(bucket, prefix):
    files = [obj.key for obj in bucket.objects.filter(Prefix=prefix)]
    return files

In [18]:
# application layer

def extract(bucket, date_list):
    files = [key for date in date_list for key in list_files_in_prefix(bucket, date)]
    df = pd.concat([read_csv_to_df(bucket, obj) for obj in files[:15]], ignore_index=True)  
    return df

def transform_report1(df, columns, arg_date):
    df = df.loc[:, columns]
    df['opening_price'] = df.sort_values(by=['Time']).groupby(['ISIN', 'Date'])['StartPrice'].transform('first')
    df['closing_price'] = df.sort_values(by=['Time']).groupby(['ISIN', 'Date'])['StartPrice'].transform('last')
    df = df.groupby(['ISIN','Date'], as_index=False).agg(opening_price_eur=('opening_price', 'min'), closing_price_eur=('closing_price', 'min'),minimun_price_eur=('MinPrice', 'min'), maximun_price_eur=('MaxPrice', 'min'), daily_traded_volumen=('TradedVolume','sum')) 
    df['prev_closing_price'] = df.sort_values(by=['Date']).groupby(['ISIN'])['closing_price_eur'].shift(1)
    df['change_prev_closing_%'] = (df['closing_price_eur'] -df['prev_closing_price']) / df ['prev_closing_price'] * 100
    df.drop(columns=['prev_closing_price'], inplace=True)
    df = df.round(decimals=2)
    #df = df[df.Date >= arg_date]
    return df

def load(bucket,df, trg_key, trg_format):
    key = trg_key + datetime.today().strftime("%Y%m%d_%H%M%S") + trg_format
    write_df_to_s3(bucket, df, key)
    return True

def etl_report1(src_bucket, trg_bucket, date_list, columns, arg_date, trg_key, trg_format):
   df = extract(src_bucket, date_list)
   df = transform_report1(df, columns, arg_date)
   df = load(trg_bucket,df, trg_key, trg_format)
   return True
    
    


In [19]:
# application layer not core

def return_date_list(bucket, arg_date, src_format, meta_key):
    min_date = datetime.strptime(arg_date, src_format).date() - timedelta(days=1)
    today = datetime.today().date()
    try:
        df_meta = read_csv_to_df(bucket, meta_key)
        dates = [(min_date + timedelta(days=x)) for x in range(0, (today - min_date).days + 1)]
        src_dates = set(pd.to_datetime(df_meta['source_date']).dt.date)
        dates_missing = set(dates[1:]) - src_dates
        if dates_missing:
            min_date = min(set(dates[1:]) - src_dates) - timedelta(days=1)
            return_dates = [date.strftime(src_format) for date in dates if date >= min_date]
            return_min_date = arg_date
        else:
            return_dates = []
            return_min_date = datetime(2200, 1, 1).date()
    except bucket.session.client('s3').exceptions.NoSuchKey:        
        return_dates = [(min_date + timedelta(days=x)).strftime(src_format) for x in range(0, (today - min_date).days + 1)]
        return_min_date = arg_date
    return return_min_date, return_dates 

In [20]:
# main funtionentry point
def main():
    # parameters /configurations
    #later read config
    arg_date = '2022-03-15'
    src_format = '%Y-%m-%d'
    src_bucket = 'xetra-1234'
    trg_bucket = 'josh-123456'
    columns=['ISIN', 'Date', 'Time', 'StartPrice', 'MaxPrice', 'MinPrice','EndPrice', 'TradedVolume']
    trg_key = 'xetra_daily_report_'
    trg_format = '.parquet'
    meta_key = "meta_file.csv" 

    # init
    s3 = boto3.resource('s3')
    bucket_src = s3.Bucket(src_bucket)
    bucket_trg = s3.Bucket(trg_bucket)

    # run application
    extract_date, date_list = return_date_list(bucket_trg, arg_date, src_format, meta_key)
    etl_report1(bucket_src, bucket_trg, date_list, columns, extract_date, trg_key, trg_format)  

In [21]:
# run
main()

C:\Users\JOSUE\AppData\Local\Temp\ipykernel_14276\3512935111.py:5: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([read_csv_to_df(bucket, obj) for obj in files[:15]], ignore_index=True)


## READING THEUPLOAD FILE

In [22]:
trg_bucket = 'josh-123456'
s3 = boto3.resource('s3')
bucket_trg = s3.Bucket(trg_bucket)
for obj in bucket_trg.objects.all():
    print(obj.key)

meta_file.csv
xetra_daily_report_20240331_122629.parquet
xetra_daily_report_20240331_122856.parquet
xetra_daily_report_20240331_123032.parquet
xetra_daily_report_20240331_141816.parquet


In [26]:
prq_obj = bucket_trg.Object(key='xetra_daily_report_20240331_141816.parquet').get().get('Body').read()
data = BytesIO(prq_obj)
df_report = pd.read_parquet(data)

In [27]:
df_report

,ISIN,Date,opening_price_eur,closing_price_eur,minimun_price_eur,maximun_price_eur,daily_traded_volumen,change_prev_closing_%
0,AT000000STR1,2022-03-15,36.40,37.55,36.40,36.40,536,NaN
1,AT00000FACC2,2022-03-15,7.88,7.66,7.61,7.61,68,NaN
2,AT0000606306,2022-03-15,13.30,13.54,12.88,12.90,19080,NaN
3,AT0000609607,2022-03-15,11.38,11.38,11.38,11.38,14,NaN
4,AT0000644505,2022-03-15,92.30,91.80,91.80,91.80,241,NaN
...,...,...,...,...,...,...,...,...
2951,XS2427363036,2022-03-15,21.49,21.49,21.49,21.49,0,NaN
2952,XS2427363895,2022-03-15,18.20,18.19,18.19,18.19,0,NaN
2953,XS2427474023,2022-03-15,25.10,25.10,25.10,25.10,0,NaN
2954,XS2434891219,2022-03-15,3.48,3.52,3.48,3.48,0,NaN
